# Download US power output PV Watts (version 8) 

## Documentation: https://developer.nrel.gov/docs/api-key/
## https://pvwatts.nrel.gov/downloads/pvwattsv5.pdf

In [1]:
import numpy as np
import scipy as sp
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [2]:
hrh_apikey = 'GNoTfD5IZWwIEz24zB5Wn0aEhDvNJSep5bwapzTI'
data_path = '/Users/hrh/Desktop/AI/erdos/rapower_data/'

In [3]:
# 1. Define PVWatts parameters and your API key
parameters = {
    'format': 'json',
    'system_capacity': 1000,
    'module_type': 0,
    'losses': 14.08,
    'array_type': 1,
    'tilt': 40,
    'azimuth': 180,
    'lat': 40.0099983215332,
    'lon':  -105.2600021362305,
    'dataset': 'tmy3',
    'radius': 25,
    'timeframe': 'monthly',
    'api_key': hrh_apikey  # Replace with your API key
}



In [4]:
url = 'https://developer.nrel.gov/api/pvwatts/v8'

# 2. Send request
response = requests.get(url, params=parameters)
data = response.json()

# 3. Convert the output data to a DataFrame
df = pd.DataFrame(data['outputs'])

print(response)

<Response [200]>


In [5]:
##### Try to get data for California on a grid with lat_delta and lon_delta resolution
# lat_delta = 0.25
# lon_delta = 0.25

# # #CONUS box
# # top = 49.3457868 # north lat
# # left = -124.7844079 # west long
# # right = -66.9513812 # east long
# # bottom =  24.7433195 # south lat

# #California box
# top    = 42.009518
# bottom = 32.534156
# left   = -124.409591 
# right  =  -114.131211


## Define list of lat lon coordinates (either for CONUS or California)

<font color ='red'> PVWatts API access currently has a limit of 1000 queries per hour. So, data has to be downloaded one chunk (size<1000) at a time </font>

In [16]:
# 1. Define a list of latitudes and longitudes for CONUS
# lats = list(np.arange(bottom, top, lat_delta))  # from southernmost to northernmost points with a 5-degree step
# lons = list(np.arange(left, right, lon_delta))  # from westernmost to easternmost points with a 5-degree step

all_data = []
##### Read lats and lons CA_grid_dataframe #############
ca_gridpath = '/Users/hrh/Desktop/AI/erdos/SolarFarmPrediction/data_including_land_cover.csv'

df_main             = pd.read_csv(ca_gridpath)
ca_grid             = df_main.loc[df_main['State']=='CA']
#ca_grid_usable      = ca_grid.loc[ca_grid['land_cover']==1.0]
ca_coords           = list(zip(ca_grid['Latitude'], ca_grid['Longitude']))
#Check if ca_coords are within california grid box
ca_lats = [item[0] for item in ca_coords] 
ca_lons = [item[1] for item in ca_coords]
(min(ca_lats),max(ca_lats)),(min(ca_lons),max(ca_lons))
#ca_coords1 = ca_coords[0:1000]
ca_coords2 = ca_coords[1000:2000]
# ca_coords3 = ca_coords[2000:3000]
# ca_coords4 = ca_coords[3000:4000]
# ca_coords5 = ca_coords[4000:]
len(ca_coords2)

1000

In [17]:
%%time
    for (lat,lon) in ca_coords2:
        parameters = {
            'format': 'json',
            'system_capacity': 1000, #1000kW = 1 MW, 1 MW or greater is considered utility-scale
            'module_type': 0,       # 0- Standard module, 1- Permium, 2-Thin film
            'losses': 14,           # Losses in percentage
            'array_type': 0,        # Open Rack: Also known as ground mount.
            'tilt': 40,
            'azimuth': 180,         # This means that the solar array is facing South in the Northeren Hemisphere
            'lat': lat,
            'lon': lon,
            'dataset': 'tmy3',      #tmy2 is 1960-1990, tmy3 is 1990-2005
            'radius': 100,            # 0-Pick the station nearest to the given (lat,lon),e.g: 50 -50 miles
            'timeframe': 'monthly',
            'api_key': hrh_apikey
        }

        url = 'https://developer.nrel.gov/api/pvwatts/v8'
        response = requests.get(url, params=parameters)
        data = response.json()
             
        # Extract and append relevant data to the all_data list
        # Extract relevant data
        print('Processing (lat,lon):',lat,lon,'\n')
        
        # Check if 'outputs' is in the data
        if 'outputs' not in data:
            print(f"No data available for latitude {lat} and longitude {lon}")
            continue

        output_data = data['outputs']
        input_data  = data['inputs']

        output_data['latitude']  = input_data['lat']
        output_data['longitude'] = input_data['lon']
        all_data.append(output_data)
        
        
# Convert all_data to a DataFrame
df = pd.DataFrame(all_data)

print(df)

Processing (lat,lon): 40.99591784656082 -123.81769687071552 

Processing (lat,lon): 41.09447848387295 -123.81769687071552 

Processing (lat,lon): 41.19303912118507 -123.81769687071552 

Processing (lat,lon): 41.2915997584972 -123.81769687071552 

Processing (lat,lon): 41.390160395809325 -123.81769687071552 

Processing (lat,lon): 41.48872103312145 -123.81769687071552 

Processing (lat,lon): 41.58728167043357 -123.81769687071552 

Processing (lat,lon): 41.6858423077457 -123.81769687071552 

Processing (lat,lon): 41.78440294505782 -123.81769687071552 

Processing (lat,lon): 41.882963582369946 -123.81769687071552 

Processing (lat,lon): 41.98152421968207 -123.81769687071552 

Processing (lat,lon): 38.82758382569408 -123.70896246712095 

Processing (lat,lon): 38.9261444630062 -123.70896246712095 

Processing (lat,lon): 39.02470510031832 -123.70896246712095 

Processing (lat,lon): 39.12326573763045 -123.70896246712095 

Processing (lat,lon): 39.22182637494257 -123.70896246712095 

Processin

In [18]:
df

,ac_monthly,poa_monthly,solrad_monthly,dc_monthly,ac_annual,solrad_annual,capacity_factor,latitude,longitude
0,"[63532.27322285928, 85982.18420047041, 108811....","[77.59338291456199, 105.4003870738621, 133.122...","[2.503012352082645, 3.764299538352218, 4.29426...","[67144.12410801073, 90412.36035440309, 114262....",1.238972e+06,4.222281,14.143519,40.99591784656082,-123.81769687071552
1,"[63532.27322285928, 85982.18420047041, 108811....","[77.59338291456199, 105.4003870738621, 133.122...","[2.503012352082645, 3.764299538352218, 4.29426...","[67144.12410801073, 90412.36035440309, 114262....",1.238972e+06,4.222281,14.143519,41.09447848387295,-123.81769687071552
2,"[63532.27322285928, 85982.18420047041, 108811....","[77.59338291456199, 105.4003870738621, 133.122...","[2.503012352082645, 3.764299538352218, 4.29426...","[67144.12410801073, 90412.36035440309, 114262....",1.238972e+06,4.222281,14.143519,41.19303912118507,-123.81769687071552
3,"[63532.27322285928, 85982.18420047041, 108811....","[77.59338291456199, 105.4003870738621, 133.122...","[2.503012352082645, 3.764299538352218, 4.29426...","[67144.12410801073, 90412.36035440309, 114262....",1.238972e+06,4.222281,14.143519,41.2915997584972,-123.81769687071552
4,"[63532.27322285928, 85982.18420047041, 108811....","[77.59338291456199, 105.4003870738621, 133.122...","[2.503012352082645, 3.764299538352218, 4.29426...","[67144.12410801073, 90412.36035440309, 114262....",1.238972e+06,4.222281,14.143519,41.390160395809325,-123.81769687071552
...,...,...,...,...,...,...,...,...,...
932,"[89834.53754588145, 92895.27475843909, 92850.5...","[109.7330518285362, 115.2283553359382, 117.917...","[3.539775865436653, 4.115298404854937, 3.80379...","[94303.42952353407, 97559.21200673214, 97775.9...",1.551509e+06,5.419652,17.711293,37.3491742660122,-121.53427439522974
933,"[89834.53754588145, 92895.27475843909, 92850.5...","[109.7330518285362, 115.2283553359382, 117.917...","[3.539775865436653, 4.115298404854937, 3.80379...","[94303.42952353407, 97559.21200673214, 97775.9...",1.551509e+06,5.419652,17.711293,37.44773490332432,-121.53427439522974
934,"[82362.22798886028, 92979.63694812944, 138943....","[99.69509192458048, 114.6672845459316, 177.377...","[3.215970707244532, 4.0952601623547, 5.7218462...","[86660.24893645362, 97640.12980180021, 145461....",1.562735e+06,5.467840,17.839440,37.54629554063645,-121.53427439522974
935,"[82362.22798886028, 92979.63694812944, 138943....","[99.69509192458048, 114.6672845459316, 177.377...","[3.215970707244532, 4.0952601623547, 5.7218462...","[86660.24893645362, 97640.12980180021, 145461....",1.562735e+06,5.467840,17.839440,37.644856177948576,-121.53427439522974


In [21]:
df.to_csv(data_path+'pvwatts_monthly_ca_1990_2005_v2.csv',index=False)

In [22]:
df1 = pd.read_csv(data_path+'pvwatts_monthly_ca_1990_2005_v2.csv')
df1

,ac_monthly,poa_monthly,solrad_monthly,dc_monthly,ac_annual,solrad_annual,capacity_factor,latitude,longitude
0,"[63532.27322285928, 85982.18420047041, 108811....","[77.59338291456199, 105.4003870738621, 133.122...","[2.503012352082645, 3.764299538352218, 4.29426...","[67144.12410801073, 90412.36035440309, 114262....",1.238972e+06,4.222281,14.143519,40.995918,-123.817697
1,"[63532.27322285928, 85982.18420047041, 108811....","[77.59338291456199, 105.4003870738621, 133.122...","[2.503012352082645, 3.764299538352218, 4.29426...","[67144.12410801073, 90412.36035440309, 114262....",1.238972e+06,4.222281,14.143519,41.094478,-123.817697
2,"[63532.27322285928, 85982.18420047041, 108811....","[77.59338291456199, 105.4003870738621, 133.122...","[2.503012352082645, 3.764299538352218, 4.29426...","[67144.12410801073, 90412.36035440309, 114262....",1.238972e+06,4.222281,14.143519,41.193039,-123.817697
3,"[63532.27322285928, 85982.18420047041, 108811....","[77.59338291456199, 105.4003870738621, 133.122...","[2.503012352082645, 3.764299538352218, 4.29426...","[67144.12410801073, 90412.36035440309, 114262....",1.238972e+06,4.222281,14.143519,41.291600,-123.817697
4,"[63532.27322285928, 85982.18420047041, 108811....","[77.59338291456199, 105.4003870738621, 133.122...","[2.503012352082645, 3.764299538352218, 4.29426...","[67144.12410801073, 90412.36035440309, 114262....",1.238972e+06,4.222281,14.143519,41.390160,-123.817697
...,...,...,...,...,...,...,...,...,...
932,"[89834.53754588145, 92895.27475843909, 92850.5...","[109.7330518285362, 115.2283553359382, 117.917...","[3.539775865436653, 4.115298404854937, 3.80379...","[94303.42952353407, 97559.21200673214, 97775.9...",1.551509e+06,5.419652,17.711293,37.349174,-121.534274
933,"[89834.53754588145, 92895.27475843909, 92850.5...","[109.7330518285362, 115.2283553359382, 117.917...","[3.539775865436653, 4.115298404854937, 3.80379...","[94303.42952353407, 97559.21200673214, 97775.9...",1.551509e+06,5.419652,17.711293,37.447735,-121.534274
934,"[82362.22798886028, 92979.63694812944, 138943....","[99.69509192458048, 114.6672845459316, 177.377...","[3.215970707244532, 4.0952601623547, 5.7218462...","[86660.24893645362, 97640.12980180021, 145461....",1.562735e+06,5.467840,17.839440,37.546296,-121.534274
935,"[82362.22798886028, 92979.63694812944, 138943....","[99.69509192458048, 114.6672845459316, 177.377...","[3.215970707244532, 4.0952601623547, 5.7218462...","[86660.24893645362, 97640.12980180021, 145461....",1.562735e+06,5.467840,17.839440,37.644856,-121.534274


In [12]:
# Convert DataFrame to GeoDataFrame
geometry = [Point(xy) for xy in zip(df1['longitude'], df1['latitude'])]
geo_df   = gpd.GeoDataFrame(df1, geometry=geometry)

In [ ]:
# Read the California county boundaries shapefile# Read the California county boundaries shapefile
ca_counties = gpd.read_file(data_path+ 'CA_counties/CA_Counties_TIGER2016.shp')

geo_df.crs  = "EPSG:4326" #Setting the crs of geo_df
#ca_counties = ca_counties.to_crs("EPSG:4326")
print('The coordinate reference system for this file:',ca_counties.crs,geo_df.crs)


# # Ensure that your data is in the same CRS as the shapefile
geo_df = geo_df.to_crs(ca_counties.crs)

# Plot the county boundaries
fig, ax = plt.subplots(figsize=(15, 15))
ca_counties.plot(ax=ax, color='white', edgecolor='black')

# Plot your data on top of the county boundaries
geo_df.plot(ax=ax, column='ac_annual', legend=True, markersize=50, cmap='OrRd', alpha=0.5)

# Add titles and labels as needed
plt.title("Annual AC Output in California by County")
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Optionally, adjust the plot limits to the extent of the shapefile or your data points
ax.set_xlim(ca_counties.bounds.minx.min(), ca_counties.bounds.maxx.max())
ax.set_ylim(ca_counties.bounds.miny.min(), ca_counties.bounds.maxy.max())

# Display the plot
plt.show()

In [ ]:
# Load US boundaries and plot
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
#us = world[world.name == "United States of America"]
ca  = world[world.name=='California']

# Plot
fig, ax = plt.subplots(figsize=(15,10))
ca.plot(ax=ax, color='lightgray')
geo_df.plot(ax=ax, column='ac_annual', legend=True, markersize=50, cmap='OrRd')  # Use the column parameter
plt.title("Annual AC Output on California")
plt.show()

In [ ]:
# {
  # "inputs": {
  #   "api_key": "DEMO_KEY",
  #   "azimuth": "180",
  #   "system_capacity": "4",
  #   "losses": "14",
  #   "array_type": "1",
  #   "module_type": "0",
  #   "gcr": "0.4",
  #   "dc_ac_ratio": "1.2",
  #   "inv_eff": "96.0",
  #   "radius": "0",
  #   "dataset": "nsrdb",
  #   "tilt": "10",
  #   "address": "boulder, co",
  #   "soiling": [
  #     12.0,
  #     4.0,
  #     45.0,
  #     23.0,
  #     9.0,
  #     99.0,
  #     67.0,
  #     12.54,
  #     54.0,
  #     9.0,
  #     0.0,
  #     7.6
  #   ],
  #   "albedo": "0.3",
  #   "bifaciality": "0.7"
  # },
  # "errors": [],
  # "warnings": [],
  # "version": "8.0.0",
  # "ssc_info": {
  #   "version": 275,
  #   "build": "Linux 64 bit GNU/C++ Oct  4 2022 03:10:07",
  #   "module": "pvwattsv8"
  # },
  # "station_info": {
  #   "lat": 40.0099983215332,
  #   "lon": -105.2600021362305,
  #   "elev": 1635.640014648438,
  #   "tz": -7.0,
  #   "location": "149190",
  #   "city": "",
  #   "state": "Colorado",
  #   "solar_resource_file": "149190.csv",
  #   "weather_data_source": "NSRDB PSM V3 GOES tmy-2020 3.2.0"
  # }